<a href="https://colab.research.google.com/github/AvisR/Drawdown/blob/main/Drawdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question - Find Nth largest drawdowns given the return series and N

Defining Drawdown - Largest negative movement in percentage. 2nd largest drawdown will be the 2nd largest negative movement in percentage. 

Drawdown = (Peak - Trough)/Peak

Return series = rets

N = N

---



In [58]:
#INPUTS - Create a list of random returns from (-5% and 10%)
#This will not be needed if we have a list of our own returns.
#II = Initial Investment

II = 1
N = 1

import random
rets = random.sample(range(-50,100),100)
rets = [x/1000 for x in rets]

print(rets)

[0.049, 0.021, 0.058, 0.035, 0.081, 0.057, 0.07, 0.038, -0.035, 0.078, -0.027, 0.082, -0.018, 0.074, 0.001, -0.005, 0.013, 0.006, 0.091, 0.076, -0.022, -0.025, -0.014, 0.088, -0.046, 0.022, 0.064, 0.032, 0.044, 0.084, 0.01, 0.012, 0.086, 0.071, 0.096, -0.019, -0.012, 0.073, -0.015, 0.05, 0.09, 0.039, 0.036, 0.008, 0.009, 0.085, -0.032, 0.052, 0.046, -0.017, 0.004, 0.024, 0.014, 0.026, 0.055, 0.023, 0.077, 0.028, 0.06, -0.01, 0.054, -0.033, 0.092, -0.007, -0.042, 0.093, 0.094, 0.066, 0.056, -0.03, 0.059, -0.043, 0.069, 0.042, 0.067, -0.024, -0.02, 0.016, -0.002, -0.031, 0.087, 0.011, 0.053, -0.028, -0.013, 0.041, 0.04, 0.018, 0.048, 0.065, 0.098, 0.017, -0.016, 0.005, 0.063, -0.036, 0.027, -0.034, 0.051, -0.045]


In [59]:
#Step 1 = Calculate value of portfolio.
#Initial investment = 1, value of the portfolio is calculated using initial investment and cumulative product.
#For eg. Value of the portfolio at the end of period 10 would be (1*(1+p1 return)*(1+p2 return)*....(1+p10 return))

cmfactor = [0]*(len(rets))

def portvalue(rets):
    mfactor = [1+x for x in rets]
    for i in range(0,len(rets)):
      if i == 0:
        cmfactor[i] = II * mfactor[i]
      else:
        cmfactor[i] = cmfactor[i-1]*mfactor[i]
      cmfactor[i] = round(cmfactor[i],4)
    return(cmfactor)

portvalue = portvalue(rets)

print(portvalue)


[1.049, 1.071, 1.1331, 1.1728, 1.2678, 1.3401, 1.4339, 1.4884, 1.4363, 1.5483, 1.5065, 1.63, 1.6007, 1.7192, 1.7209, 1.7123, 1.7346, 1.745, 1.9038, 2.0485, 2.0034, 1.9533, 1.926, 2.0955, 1.9991, 2.0431, 2.1739, 2.2435, 2.3422, 2.5389, 2.5643, 2.5951, 2.8183, 3.0184, 3.3082, 3.2453, 3.2064, 3.4405, 3.3889, 3.5583, 3.8785, 4.0298, 4.1749, 4.2083, 4.2462, 4.6071, 4.4597, 4.6916, 4.9074, 4.824, 4.8433, 4.9595, 5.0289, 5.1597, 5.4435, 5.5687, 5.9975, 6.1654, 6.5353, 6.4699, 6.8193, 6.5943, 7.201, 7.1506, 6.8503, 7.4874, 8.1912, 8.7318, 9.2208, 8.9442, 9.4719, 9.0646, 9.6901, 10.0971, 10.7736, 10.515, 10.3047, 10.4696, 10.4487, 10.1248, 11.0057, 11.1268, 11.7165, 11.3884, 11.2404, 11.7013, 12.1694, 12.3884, 12.983, 13.8269, 15.1819, 15.44, 15.193, 15.269, 16.2309, 15.6466, 16.0691, 15.5228, 16.3145, 15.5803]


In [60]:
# Step 2 = Find all difference. 
# The idea is to create a matrix which contains differences betweem all portfolio value. 
# For eg - The first row of the matrix will contain difference between initial investment and the subsequent values. 
# This can be optimized by running the for loop in reverse(i.e from 99 to 0) and stopping the iteration as soon we find a future drawdown greater than the current drawdown.  
# For eg. Value at t0 = 1. Value at time 10 = 1.3 Value at time 23 = .5. The drawdown from time 10 will always be greater than the drawdown from t0. So we need to evaluate only till t9 for t0.
# This is current done by running a for loop where I compare each element[i][a] with elements after in the same row[i][a+]. If element that comes after[i][a+] are smaller(larger drawdown), the element will reset to 0 now and be removed in Step 4.  


#Matrix of differences

ddmatrix = [[0 for i in range(len(rets))] for a in range(len(rets))]

def diffmat(rets):
    for i in range(len(rets)):
      if i == 0:
        for a in range(len(rets)):
         ddmatrix[i][a]= (portvalue[a]-II)/II
      else:
        for a in range(len(rets)):
         ddmatrix[i][a]= (portvalue[a]-portvalue[i-1])/portvalue[i-1]
    return(ddmatrix)

ddmatrix = diffmat(rets)

#Making it upper triangle matrix. 
for i in range(len(ddmatrix)):
  for a in range(len(ddmatrix)):
    if a < i:
      ddmatrix[i][a] = 0
    else:
      ddmatrix[i][a] = round(ddmatrix[i][a],4)
    for b in range(a+1,len(ddmatrix)):
      if ddmatrix[i][a] > ddmatrix[i][b]:
        ddmatrix[i][a] = 0


In [61]:
#Step 3 = Find the largest drawdown for each date. 

mddmatrix = [[0 for i in range(3)] for a in range(len(rets))]

def min_value(ddmatrix):
      l = len(ddmatrix)
      minlist = [min((ddmatrix)[i]) for i in range(l)]
      mdd = minlist
      return (minlist)

mdd = min_value(ddmatrix)

print(list(mdd))

[0, 0, 0, 0, 0, 0, 0, 0, -0.035, 0, -0.027, 0, -0.018, 0, -0.004, -0.005, 0, 0, 0, 0, -0.0598, -0.0386, -0.014, 0, -0.046, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.0308, -0.012, 0, -0.015, 0, 0, 0, 0, 0, 0, 0, -0.032, 0, 0, -0.017, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.01, 0, -0.033, 0, -0.0487, -0.042, 0, 0, 0, 0, -0.03, 0, -0.043, 0, 0, 0, -0.0602, -0.0371, -0.0175, -0.0329, -0.031, 0, 0, 0, -0.0406, -0.013, 0, 0, 0, 0, 0, 0, 0, -0.016, 0, 0, -0.0436, -0.0079, -0.034, 0, -0.045]


In [62]:
#Step 4 - Build MDD Matrix(Maximum drawdown for each date along with the time period). 
#Remove 0s and overlapping drawdowns and return Nth drawdown. 

mddmatrix = [[0 for i in range(3)] for a in range(len(rets))]

#Build MDD matrix and remove Zero drawdowns
for x in range(len(mddmatrix)):
  mddmatrix[x][0] = mdd[x]
  for a in range(len(rets)):
    if ddmatrix[x][a] == mddmatrix[x][0]:
       mddmatrix[x][1] = x
       mddmatrix[x][2] = a+1
    if mddmatrix[x][0] >= 0: 
       mddmatrix[x][1] = 0
       mddmatrix[x][2] = 0

mddmatrix = ([i for i in mddmatrix if any(i)])
mddmatrix.sort()

#Remove overlapping drawdowns
def unique(mddmat): 
    found = set()
    for item in mddmat:
        if item[2] not in found:
            yield item
            found.add(item[2])

mddmatrix = list(unique(mddmatrix))

#nth drawdown
def ndraw(N):
  if N > len(mddmatrix):
    return(print("We do not have", N, "drawdown"))
  elif mddmatrix[1] == 0:
    return(print("We do not have", N, "drawdown"))
  else:
    return(mddmatrix[N-1])

ndrawdown = ndraw(N)

print(N, "th largest drawdown is", ndrawdown[0], "and it occurs between", ndrawdown[1], "and", ndrawdown[2])
print(mddmatrix)

1 th largest drawdown is -0.0602 and it occurs between 75 and 80
[[-0.0602, 75, 80], [-0.0598, 20, 23], [-0.0487, 63, 65], [-0.046, 24, 25], [-0.045, 99, 100], [-0.0436, 95, 98], [-0.043, 71, 72], [-0.0406, 83, 85], [-0.035, 8, 9], [-0.033, 61, 62], [-0.032, 46, 47], [-0.0308, 35, 37], [-0.03, 69, 70], [-0.027, 10, 11], [-0.018, 12, 13], [-0.017, 49, 50], [-0.016, 92, 93], [-0.015, 38, 39], [-0.01, 59, 60], [-0.005, 15, 16]]
